<a href="https://colab.research.google.com/github/JRK-007/realtime-parking-pricing/blob/main/Dynamic_Pricing_for_Urban_Parking_Lots_(Google_Colab_Template).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Dynamic Pricing for Urban Parking Lots - Google Colab Template

This notebook provides a template for implementing a dynamic pricing system
for urban parking lots, as outlined in the Summer Analytics 2025 Capstone Project.

It includes:
1.  Setup and necessary imports (pandas, numpy, bokeh, pathway - simulated).
2.  Helper functions for calculations like Haversine distance and data normalization.
3.  Implementation of three pricing models:
    -   Model 1: Baseline Linear Model
    -   Model 2: Demand-Based Price Function
    -   Model 3: Competitive Pricing Model (Simplified)
4.  A mock data generator to simulate real-time data streams.
5.  Bokeh visualization setup for real-time pricing plots.
6.  A simulated real-time loop to demonstrate the pricing updates and visualizations.

**IMPORTANT:**
-   This code simulates real-time data using a loop and mock data. You will need
    to replace the mock data generation and the simulation loop with actual
    Pathway integration (`pw.io.csv.read`, `pw.run`, etc.) once you have the
    provided Pathway sample notebook and `dataset.csv`.
-   The pricing model coefficients are placeholders and require tuning based on
    your analysis of the actual `dataset.csv`.
-   The competitive pricing model (Model 3) is simplified for demonstration.
    A full implementation would require more sophisticated logic for identifying
    nearby lots and fetching their real-time prices within the Pathway stream.
"""

# 1. Setup and Imports
# Import necessary libraries for data manipulation, time simulation, random number generation,
# mathematical operations, and visualization.
import pandas as pd
import numpy as np
import time
import random
from math import radians, sin, cos, sqrt, atan2

# Bokeh for real-time visualizations:
# `figure` for creating plots.
# `show` for displaying plots.
# `curdoc` (for Bokeh server applications, less critical for static Colab output here).
# `output_notebook` to render Bokeh plots directly in the Jupyter/Colab notebook.
# `push_notebook` for updating plots dynamically in a live notebook session (requires `show` with `notebook_handle=True`).
# `ColumnDataSource` to hold data for Bokeh plots, allowing for streaming updates.
# `LinearAxis`, `Range1d` for adding secondary axes to plots.
# `column`, `row` for arranging multiple plots in a layout.
# `Category10` for a palette of distinct colors for multiple plots.
from bokeh.plotting import figure, show, curdoc
from bokeh.io import output_notebook, push_notebook
from bokeh.models import ColumnDataSource, LinearAxis, Range1d
from bokeh.layouts import column, row
from bokeh.palettes import Category10

# Pathway (simulated import - actual Pathway setup will differ)
# The following commented-out block is a placeholder for Pathway integration.
# In a real scenario, you would uncomment 'import pathway as pw' and ensure Pathway is installed.
# The MockPathway class is included for demonstration purposes if Pathway is not installed,
# allowing the code structure to be understood without a full Pathway environment.
# try:
#     import pathway as pw
# except ImportError:
#     print("Pathway not found. Please install it: pip install pathway")
#     print("Proceeding with simulated Pathway environment.")
#     # Mock Pathway objects for demonstration if Pathway is not installed
#     class MockPathway:
#         class io:
#             class csv:
#                 def read(self, *args, **kwargs):
#                     print("Mock Pathway CSV read called.")
#                     return self # Return self to allow chaining
#             class kafka:
#                 def read(self, *args, **kwargs):
#                     print("Mock Pathway Kafka read called.")
#                     return self # Return self to allow chaining
#         def run_until_exception(self, *args, **kwargs):
#             print("Mock Pathway run_until_exception called.")
#         def run(self, *args, **kwargs):
#             print("Mock Pathway run called.")
#         def select(self, *args, **kwargs):
#             print("Mock Pathway select called.")
#             return self
#         def transform(self, *args, **kwargs):
#             print("Mock Pathway transform called.")
#             return self
#         def stateful_map(self, *args, **kwargs):
#             print("Mock Pathway stateful_map called.")
#             return self
#         def map(self, *args, **kwargs):
#             print("Mock Pathway map called.")
#             return self
#         def groupby(self, *args, **kwargs):
#             print("Mock Pathway groupby called.")
#             return self
#         def reduce(self, *args, **kwargs):
#             print("Mock Pathway reduce called.")
#             return self
#         def debug_print(self, *args, **kwargs):
#             print("Mock Pathway debug_print called.")
#             return self
#         def from_dataframe(self, df):
#             print("Mock Pathway from_dataframe called.")
#             return self
#     pw = MockPathway()

# Ensure Bokeh plots appear directly within the Google Colab notebook output.
output_notebook()

# 2. Global Constants
# Define constants that will be used throughout the pricing models.
BASE_PRICE = 10.0  # The initial or reference price for parking.
MIN_PRICE_FACTOR = 0.5 # Minimum allowed price as a factor of the base price (e.g., 0.5 * $10 = $5).
MAX_PRICE_FACTOR = 2.0 # Maximum allowed price as a factor of the base price (e.g., 2.0 * $10 = $20).

# Define a few mock parking spaces for simulation purposes.
# In a real application, this data would be loaded from `dataset.csv` via Pathway.
# Each parking space has a unique ID, latitude, longitude, capacity, and an initial price.
MOCK_PARKING_SPACES = {
    'P1': {'lat': 11.0045, 'lon': 76.9616, 'capacity': 100, 'initial_price': BASE_PRICE},
    'P2': {'lat': 11.0050, 'lon': 76.9620, 'capacity': 80, 'initial_price': BASE_PRICE},
    'P3': {'lat': 11.0030, 'lon': 76.9600, 'capacity': 120, 'initial_price': BASE_PRICE},
    'P4': {'lat': 11.0060, 'lon': 76.9630, 'capacity': 90, 'initial_price': BASE_PRICE},
    'P5': {'lat': 11.0040, 'lon': 76.9590, 'capacity': 70, 'initial_price': BASE_PRICE}
}

# Initialize the state for each mock parking space.
# This dictionary will hold the current operational data and the last calculated price for each lot.
# Random initial values are assigned for simulation.
parking_lot_states = {
    pid: {
        'current_price': MOCK_PARKING_SPACES[pid]['initial_price'], # The price currently being charged.
        'occupancy': random.randint(20, MOCK_PARKING_SPACES[pid]['capacity']), # Current number of vehicles parked.
        'queue_length': random.randint(0, 5), # Number of vehicles waiting to enter.
        'traffic_congestion_level': random.uniform(0.1, 0.9), # A normalized value (e.g., 0-1) indicating traffic.
        'is_special_day': random.choice([0, 1]), # Binary indicator: 1 if it's a holiday/event, 0 otherwise.
        'incoming_vehicle_type': random.choice(['car', 'bike', 'truck']), # Type of vehicle arriving.
        'lat': MOCK_PARKING_SPACES[pid]['lat'], # Latitude of the parking lot.
        'lon': MOCK_PARKING_SPACES[pid]['lon'], # Longitude of the parking lot.
        'capacity': MOCK_PARKING_SPACES[pid]['capacity'] # Maximum capacity of the parking lot.
    } for pid in MOCK_PARKING_SPACES
}


# 3. Helper Functions
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth (given by latitude and longitude)
    using the Haversine formula. This is crucial for the competitive pricing model.

    Args:
        lat1 (float): Latitude of the first point (in degrees).
        lon1 (float): Longitude of the first point (in degrees).
        lat2 (float): Latitude of the second point (in degrees).
        lon2 (float): Longitude of the second point (in degrees).
    Returns:
        float: The distance between the two points in kilometers.
    """
    R = 6371  # Radius of Earth in kilometers

    # Convert latitude and longitude from degrees to radians for trigonometric calculations.
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate the differences in longitude and latitude.
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Apply the Haversine formula.
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def normalize_value(value, min_val, max_val):
    """
    Normalizes a given value to a range between 0 and 1 based on specified minimum and maximum bounds.
    This is useful for scaling features before using them in demand functions.

    Args:
        value (float): The value to normalize.
        min_val (float): The minimum possible value in the original range.
        max_val (float): The maximum possible value in the original range.
    Returns:
        float: The normalized value, clamped between 0 and 1. Returns 0.0 if min_val equals max_val
               to prevent division by zero.
    """
    if max_val == min_val:
        return 0.0 # Avoid division by zero if range is zero
    return (value - min_val) / (max_val - min_val)

def get_vehicle_type_weight(vehicle_type):
    """
    Assigns a numerical weight to different vehicle types. This weight can be used
    in the demand function to reflect varying demand or impact based on vehicle type.

    Args:
        vehicle_type (str): The type of incoming vehicle ('car', 'bike', or 'truck').
    Returns:
        float: A numerical weight corresponding to the vehicle type. Defaults to 1.0
               if the type is not recognized.
    """
    # Example weights: cars have base weight, bikes might take less space/generate less demand,
    # trucks might be more demanding or indicate commercial activity.
    weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.3}
    return weights.get(vehicle_type, 1.0) # Return default if vehicle_type is not in weights

# 4. Pricing Logic Implementation (Core Requirements)

# Model 1: Baseline Linear Model
def calculate_price_model1(prev_price, occupancy, capacity):
    """
    Implements the Baseline Linear Model, which is a simple reference point.
    The price at the next time step ($Price_{t+1}$) is a linear function of the
    previous price ($Price_t$) and the current occupancy rate.

    Formula: $Price_{t+1} = Price_t + \alpha \times (Occupancy / Capacity)$

    Args:
        prev_price (float): The price from the immediate previous time step.
        occupancy (int): Current number of vehicles occupying the parking lot.
        capacity (int): Maximum capacity of the parking lot.
    Returns:
        float: The calculated new price, clamped within the global min/max price factors.
    """
    alpha = 5.0  # Coefficient determining the impact of occupancy rate on price change.
                 # A higher alpha means price increases more sharply with higher occupancy.
                 # This value needs to be tuned for realistic behavior.
    occupancy_rate = occupancy / capacity if capacity > 0 else 0.0 # Calculate current occupancy rate.
    new_price = prev_price + alpha * occupancy_rate # Apply the linear pricing formula.

    # Ensure the calculated price stays within predefined bounds (0.5x to 2x BASE_PRICE).
    # This prevents erratic or unrealistic price fluctuations.
    new_price = max(BASE_PRICE * MIN_PRICE_FACTOR, min(BASE_PRICE * MAX_PRICE_FACTOR, new_price))
    return new_price

# Model 2: Demand-Based Price Function
def calculate_price_model2(base_price, occupancy, capacity, queue_length, traffic_level, is_special_day, incoming_vehicle_type):
    """
    Implements a more advanced demand-based pricing model.
    First, it constructs a mathematical demand function using various features.
    Then, it adjusts the base price based on this calculated demand.

    Demand Function (Example):
    $Demand = \alpha_{occ} \times (Occupancy/Capacity) + \beta_{queue} \times QueueLength +
             \gamma_{traffic} \times Traffic + \delta_{special} \times IsSpecialDay +
             \epsilon_{vehicle} \times VehicleTypeWeight$

    Price Adjustment Formula:
    $Price = Base Price \times (1 + \lambda \times NormalizedDemand)$

    Args:
        base_price (float): The base price for the parking lot.
        occupancy (int): Current number of vehicles parked.
        capacity (int): Maximum capacity of the parking lot.
        queue_length (int): Number of vehicles waiting for entry.
        traffic_level (float): Nearby traffic congestion level (e.g., 0-1, higher means more congestion).
        is_special_day (int): Binary indicator (1 for special day, 0 otherwise).
        incoming_vehicle_type (str): Type of incoming vehicle ('car', 'bike', 'truck').
    Returns:
        tuple: A tuple containing (calculated new price, raw demand value).
    """
    occupancy_rate = occupancy / capacity if capacity > 0 else 0.0 # Calculate current occupancy rate.
    vehicle_type_weight = get_vehicle_type_weight(incoming_vehicle_type) # Get weight for vehicle type.

    # Define coefficients for the demand function. These coefficients determine the
    # sensitivity of the calculated demand to each input feature.
    # These values are crucial for model behavior and require careful tuning based on data.
    alpha_occ = 3.0   # Impact of occupancy rate: higher occupancy -> higher demand.
    beta_queue = 0.8  # Impact of queue length: longer queue -> higher demand.
    gamma_traffic = 1.5 # Impact of traffic congestion: more traffic -> potentially higher demand (people looking for parking).
    delta_special = 2.5 # Impact of special day: special events/holidays -> significantly higher demand.
    epsilon_vehicle = 0.5 # Impact of vehicle type weight on demand.

    # Construct the raw demand value by summing weighted features.
    raw_demand = (alpha_occ * occupancy_rate +
                  beta_queue * queue_length +
                  gamma_traffic * traffic_level +
                  delta_special * is_special_day +
                  epsilon_vehicle * vehicle_type_weight)

    # Normalize the raw demand to a 0-1 range.
    # The `min_raw_demand` and `max_raw_demand` should ideally be derived from
    # statistical analysis of your `dataset.csv` to reflect realistic bounds.
    # For simulation, assumed ranges are used here.
    min_raw_demand = 0.0 # Assumed minimum possible demand.
    # Assumed maximum possible demand, calculated by taking max values for each feature.
    max_raw_demand = (alpha_occ * 1.0 + beta_queue * 10 + gamma_traffic * 1.0 +
                      delta_special * 1 + epsilon_vehicle * 1.3)
    normalized_demand = normalize_value(raw_demand, min_raw_demand, max_raw_demand)

    # Lambda coefficient ($\lambda$) for price adjustment. This determines how strongly
    # normalized demand influences the final price. Tune this value.
    lambda_val = 0.4

    # Calculate the new price based on the base price and normalized demand.
    new_price = base_price * (1 + lambda_val * normalized_demand)

    # Ensure the calculated price stays within predefined bounds to maintain stability.
    new_price = max(base_price * MIN_PRICE_FACTOR, min(base_price * MAX_PRICE_FACTOR, new_price))

    return new_price, raw_demand # Return both price and raw demand for analysis/plotting.

# Model 3: Competitive Pricing Model
def calculate_price_model3(current_lot_id, current_lot_data, all_parking_lot_states, price_model2):
    """
    Implements the Competitive Pricing Model, which adds real-world competition
    by considering the prices and status of nearby parking spaces.

    Args:
        current_lot_id (str): The unique identifier of the current parking lot.
        current_lot_data (dict): A dictionary containing the current operational data
                                 (occupancy, lat, lon, etc.) for the current lot.
        all_parking_lot_states (dict): A dictionary containing the current states
                                       (including prices) of ALL parking lots. This is
                                       essential for competitive analysis.
        price_model2 (float): The price calculated by Model 2 for the current parking lot.
                              This serves as the baseline for competitive adjustments.
    Returns:
        float: The final adjusted price after considering competitive factors,
               clamped within the global min/max price factors.
    """
    final_price = price_model2 # Start with the price from Model 2.
    nearby_competitors = []
    proximity_threshold_km = 0.5 # Defines the radius (in km) to consider a lot "nearby". Tune this.

    # 1. Calculate geographic proximity of nearby parking spaces.
    # Iterate through all other parking lots to find competitors within the threshold.
    for other_lot_id, other_lot_data in all_parking_lot_states.items():
        if other_lot_id != current_lot_id: # Ensure not comparing a lot to itself.
            dist = haversine_distance(
                current_lot_data['lat'], current_lot_data['lon'],
                other_lot_data['lat'], other_lot_data['lon']
            )
            if dist <= proximity_threshold_km:
                # If within proximity, add competitor's relevant data.
                nearby_competitors.append({
                    'id': other_lot_id,
                    'distance': dist,
                    'price': other_lot_data['current_price'], # Get their current price.
                    'occupancy_rate': other_lot_data['occupancy'] / other_lot_data['capacity'] # Get their occupancy.
                })

    if not nearby_competitors:
        return final_price # If no competitors are found nearby, return the Model 2 price unchanged.

    # Sort competitors by distance (closest ones first). This might be useful for prioritizing.
    nearby_competitors.sort(key=lambda x: x['distance'])

    # 2. Implement competitive logic based on scenarios.
    current_occupancy_rate = current_lot_data['occupancy'] / current_lot_data['capacity']

    # Scenario 1: Your lot is full/near full AND nearby lots are cheaper.
    # In this case, you might want to slightly reduce your price to remain competitive
    # or encourage customers to consider waiting, or conceptually suggest rerouting.
    if current_occupancy_rate > 0.90: # If lot is more than 90% full.
        for comp in nearby_competitors:
            if comp['price'] < final_price * 0.95: # If a nearby competitor is significantly cheaper (e.g., 5% cheaper).
                final_price *= 0.98 # Apply a small price reduction (e.g., 2%).
                # In a real system, you might also trigger a "rerouting suggestion" message here.
                # print(f"  {current_lot_id}: Near full, nearby {comp['id']} is cheaper. Reducing price.")
                break # Act on the first significantly cheaper competitor found.

    # Scenario 2: If nearby lots are generally more expensive.
    # This allows you to potentially increase your price while still being attractive
    # compared to the competition.
    if nearby_competitors:
        avg_comp_price = np.mean([comp['price'] for comp in nearby_competitors]) # Calculate average price of competitors.
        if avg_comp_price > final_price * 1.1: # If average competitor price is 10% higher than yours.
            final_price *= 1.02 # Apply a small price increase (e.g., 2%).
            # print(f"  {current_lot_id}: Nearby competitors expensive. Slightly increasing price.")

    # Apply bounds again after competitive adjustments to ensure the final price
    # remains within the globally defined min/max factors.
    final_price = max(BASE_PRICE * MIN_PRICE_FACTOR, min(BASE_PRICE * MAX_PRICE_FACTOR, final_price))
    return final_price


# 5. Mock Data Generator (Simulates real-time stream)
def generate_mock_data_for_parking_lot(parking_lot_id, current_state):
    """
    Generates mock real-time data for a single parking lot.
    This function simulates the continuous arrival of new data points,
    mimicking a real-time data stream that Pathway would typically ingest.
    The changes are randomized to simulate dynamic conditions.

    Args:
        parking_lot_id (str): The ID of the parking lot for which to generate data.
        current_state (dict): The current operational state of the parking lot.
    Returns:
        dict: A dictionary containing new simulated data for the parking lot.
    """
    capacity = current_state['capacity']

    # Simulate occupancy changes:
    # Randomly adjust current occupancy, ensuring it stays within 0 and capacity.
    # Introduce a small chance of larger influx/outflux to simulate events.
    current_occupancy = current_state['occupancy']
    change = random.randint(-10, 10) # Random change in occupancy (e.g., +/- 10 vehicles).
    new_occupancy = max(0, min(capacity, current_occupancy + change))
    if random.random() < 0.1: # 10% chance of a significant event.
        new_occupancy = max(0, min(capacity, new_occupancy + random.randint(-30, 30))) # Larger change.

    # Simulate changes for other features:
    new_queue_length = max(0, min(15, current_state['queue_length'] + random.randint(-1, 2))) # Queue length fluctuates.
    new_traffic_congestion_level = max(0.1, min(0.9, current_state['traffic_congestion_level'] + random.uniform(-0.1, 0.1))) # Traffic fluctuates.
    new_is_special_day = random.choices([0, 1], weights=[0.95, 0.05], k=1)[0] # Low probability of a special day.
    new_incoming_vehicle_type = random.choices(['car', 'bike', 'truck'], weights=[0.7, 0.2, 0.1], k=1)[0] # Vehicle type distribution.

    return {
        'parking_lot_id': parking_lot_id,
        'latitude': current_state['lat'],
        'longitude': current_state['lon'],
        'capacity': capacity,
        'occupancy': new_occupancy,
        'queue_length': new_queue_length,
        'traffic_congestion_level': new_traffic_congestion_level,
        'is_special_day': new_is_special_day,
        'incoming_vehicle_type': new_incoming_vehicle_type,
        'timestamp': time.time() # Record the current timestamp for plotting.
    }

# 6. Bokeh Visualization Setup
# Create a `ColumnDataSource` for each parking lot. These sources will be updated
# with new data points in real-time, and Bokeh plots will automatically reflect these changes.
sources = {pid: ColumnDataSource(data=dict(time=[], price=[], occupancy=[], demand=[]))
           for pid in MOCK_PARKING_SPACES}

# Dictionaries to store Bokeh plot figures and their line handles.
plots = {}
handles = {} # Handles can be used with `push_notebook` for highly interactive updates.
# Generate a palette of distinct colors for each parking lot's plot.
colors = Category10[len(MOCK_PARKING_SPACES)]

# Loop through each parking space to create a dedicated Bokeh plot.
for i, (pid, _) in enumerate(MOCK_PARKING_SPACES.items()):
    p = figure(
        x_axis_label='Time (seconds)', # X-axis label.
        y_axis_label='Price ($)',     # Primary Y-axis label for price.
        title=f'Real-time Price for Parking Lot {pid}', # Plot title.
        height=300, # Height of each individual plot.
        width=800,  # Width of each individual plot.
        x_axis_type='datetime', # Configure x-axis to display time in a readable format.
        tools="pan,wheel_zoom,box_zoom,reset,save" # Enable interactive tools for the plot.
    )

    # Add a second y-axis on the right side for displaying Occupancy Rate.
    # This allows comparing price and occupancy on the same plot.
    p.extra_y_ranges = {"occupancy_range": Range1d(start=0, end=1.1)} # Define the range for the secondary axis.
    p.add_layout(LinearAxis(y_range_name="occupancy_range", axis_label="Occupancy Rate"), 'right')

    # Plot the price line for the current parking lot.
    price_line = p.line(
        x='time',
        y='price',
        source=sources[pid], # Link to the ColumnDataSource for real-time updates.
        line_width=2,
        color=colors[i], # Assign a unique color from the palette.
        legend_label=f'{pid} Price' # Label for the legend.
    )

    # Plot the occupancy rate line on the secondary y-axis.
    occupancy_line = p.line(
        x='time',
        y='occupancy',
        source=sources[pid],
        line_width=1,
        color='gray',
        line_dash='dashed', # Use a dashed line for distinction.
        legend_label=f'{pid} Occupancy Rate',
        y_range_name="occupancy_range" # Assign to the secondary y-axis.
    )

    p.legend.location = "top_left" # Position the legend.
    p.legend.click_policy="hide" # Allow users to hide/show lines by clicking their legend entries.
    plots[pid] = p # Store the figure object.
    handles[pid] = {'price': price_line, 'occupancy': occupancy_line} # Store line handles (optional, for advanced updates).

# Arrange all individual parking lot plots vertically in a single column layout.
layout = column(*plots.values())

# Show the initial plots.
# In a true real-time Bokeh application with a server, you would use `curdoc().add_root(layout)`.
# For Google Colab, `show(layout)` will render the plots. If you want dynamic updates
# within a single output cell, you would typically use `show(layout, notebook_handle=True)`
# and then `push_notebook(handle=plot_handle)` inside your update loop.
# For simplicity in this template, we re-show the plot at the end of the simulation,
# which will redraw it with all accumulated data.
# plot_handle = show(layout, notebook_handle=True) # Uncomment this and use push_notebook for live updates.


# 7. Simulated Real-Time Loop
print("Starting real-time pricing simulation...")
print("This simulation runs for a few iterations. In a real scenario, Pathway would handle continuous streaming.")

num_iterations = 50 # Total number of simulated time steps or data points.
update_interval_seconds = 1 # The delay between each simulated data point update.

# Record the start time to calculate relative time for the x-axis of plots.
start_time = time.time()

# Main simulation loop. This loop mimics Pathway's continuous data processing.
for i in range(num_iterations):
    print(f"\n--- Iteration {i+1} ---")
    current_time_offset = time.time() - start_time # Time elapsed since the simulation started.

    # Process data and update prices for each parking lot.
    for pid in MOCK_PARKING_SPACES:
        # 1. Generate new mock data for the current parking lot.
        # This simulates a new data record arriving in the stream.
        new_data_point = generate_mock_data_for_parking_lot(pid, parking_lot_states[pid])

        # 2. Update the internal state of the current parking lot with the newly generated data.
        parking_lot_states[pid].update({
            'occupancy': new_data_point['occupancy'],
            'queue_length': new_data_point['queue_length'],
            'traffic_congestion_level': new_data_point['traffic_congestion_level'],
            'is_special_day': new_data_point['is_special_day'],
            'incoming_vehicle_type': new_data_point['incoming_vehicle_type']
        })

        # 3. Calculate prices using the three defined models.
        # Model 1: Baseline Linear Model calculation.
        price_m1 = calculate_price_model1(
            parking_lot_states[pid]['current_price'], # Uses the previously calculated price for this lot.
            parking_lot_states[pid]['occupancy'],
            parking_lot_states[pid]['capacity']
        )

        # Model 2: Demand-Based Price Function calculation.
        # This model uses the BASE_PRICE as its starting point for calculation.
        price_m2, raw_demand = calculate_price_model2(
            BASE_PRICE,
            parking_lot_states[pid]['occupancy'],
            parking_lot_states[pid]['capacity'],
            parking_lot_states[pid]['queue_length'],
            parking_lot_states[pid]['traffic_congestion_level'],
            parking_lot_states[pid]['is_special_day'],
            parking_lot_states[pid]['incoming_vehicle_type']
        )

        # Model 3: Competitive Pricing Model calculation.
        # This model takes the price from Model 2 as its base and adjusts it based on competitors.
        # It requires the state of all parking lots to assess competition.
        price_m3 = calculate_price_model3(
            pid,
            parking_lot_states[pid],
            parking_lot_states, # Pass the entire dictionary of parking lot states for competitive analysis.
            price_m2
        )

        # 4. Choose which model's price to use for the final output.
        # In this template, Model 3's output is chosen if implemented, otherwise Model 2's.
        final_price = price_m3 # You can change this to price_m1 or price_m2 if you want to test specific models.

        # 5. Update the current price in the parking lot's state.
        parking_lot_states[pid]['current_price'] = final_price

        # 6. Prepare the new data point for Bokeh visualization.
        # Time is converted to milliseconds, as Bokeh's datetime axis expects this format.
        new_bokeh_data = dict(
            time=[(start_time + current_time_offset) * 1000], # Convert seconds to milliseconds.
            price=[final_price],
            occupancy=[parking_lot_states[pid]['occupancy'] / parking_lot_states[pid]['capacity']], # Calculate occupancy rate for plotting.
            demand=[raw_demand] # Include raw demand for potential future plotting or debugging.
        )

        # 7. Stream the new data to the corresponding Bokeh ColumnDataSource.
        # `rollover=50` keeps only the last 50 data points, preventing the plot from becoming too dense.
        sources[pid].stream(new_bokeh_data, rollover=50)

        # Print current status to the console for real-time monitoring.
        print(f"  Lot {pid}: Occupancy={parking_lot_states[pid]['occupancy']}/{parking_lot_states[pid]['capacity']}"
              f" ({parking_lot_states[pid]['occupancy'] / parking_lot_states[pid]['capacity']:.2f}), "
              f"Queue={parking_lot_states[pid]['queue_length']}, "
              f"Traffic={parking_lot_states[pid]['traffic_congestion_level']:.2f}, "
              f"SpecialDay={parking_lot_states[pid]['is_special_day']}, "
              f"Vehicle={parking_lot_states[pid]['incoming_vehicle_type']}, "
              f"RawDemand={raw_demand:.2f}, Price={final_price:.2f}")

    # Update the Bokeh plot.
    # If using `show(notebook_handle=True)` and `push_notebook(handle=plot_handle)`:
    # `push_notebook(handle=plot_handle)` would send updates to the existing plot.
    # For this simpler setup, we only call `show(layout)` once at the very end of the simulation.
    # If you want to see updates during the loop, you could call `show(layout)` every few iterations,
    # but it will redraw the entire plot each time, which can be slow.
    if i == num_iterations - 1: # Only show the final plot after all iterations are complete.
        show(layout)

    # Pause for the defined interval to simulate real-time data arrival delay.
    time.sleep(update_interval_seconds)

print("\nSimulation finished.")
print("Remember to replace the mock data and simulation loop with your actual Pathway integration.")
print("Tune the model coefficients for optimal pricing behavior based on your dataset.")


Starting real-time pricing simulation...
This simulation runs for a few iterations. In a real scenario, Pathway would handle continuous streaming.

--- Iteration 1 ---
  Lot P1: Occupancy=53/100 (0.53), Queue=6, Traffic=0.53, SpecialDay=0, Vehicle=bike, RawDemand=7.54, Price=11.93
  Lot P2: Occupancy=57/80 (0.71), Queue=3, Traffic=0.28, SpecialDay=0, Vehicle=bike, RawDemand=5.30, Price=11.36
  Lot P3: Occupancy=60/120 (0.50), Queue=5, Traffic=0.10, SpecialDay=0, Vehicle=car, RawDemand=6.15, Price=11.57
  Lot P4: Occupancy=57/90 (0.63), Queue=5, Traffic=0.59, SpecialDay=0, Vehicle=bike, RawDemand=7.13, Price=11.82
  Lot P5: Occupancy=20/70 (0.29), Queue=2, Traffic=0.89, SpecialDay=0, Vehicle=car, RawDemand=4.29, Price=11.10

--- Iteration 2 ---
  Lot P1: Occupancy=51/100 (0.51), Queue=7, Traffic=0.54, SpecialDay=0, Vehicle=car, RawDemand=8.44, Price=12.16
  Lot P2: Occupancy=34/80 (0.42), Queue=2, Traffic=0.19, SpecialDay=1, Vehicle=truck, RawDemand=6.31, Price=11.61
  Lot P3: Occupancy